This notebook is an attempt to reduce the racial disparity in machine learning algorithms. It does this by finding instances where one race is mentioned specifically and adding a new instance where the text and metadata are the same except the race has been changed. To reduce complexity for this test other ethnicities are not included at the moment. If this technique actually works they will be added.

In [233]:
import pandas as pd
import re

In [234]:
df = pd.read_csv('C:/Users/HMISYS/Documents/GitHub/D4D/hate_speech_detector/data/twitter-hate-speech2.csv')

In [235]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [236]:
df.describe()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


We're working with 24784 samples.

In [117]:
# Don't truncate the text
pd.set_option('display.max_colwidth', -1)
# Show lots of examples
pd.options.display.max_rows = 400

Tweets that mention a specific race, such as those shown below, are likely to cause a racial disparity in a machine learning algorithm.

In [34]:
df[df['tweet'].str.contains("black people")]['tweet']

184      "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPLE https://t.co/RNvD2nLCDR" This is why there's black people and niggers                                           
1465     &#8220;@PlMPCESS: In the fashion world "urban" means "stolen from black people" its ghetto on us, and urban on them haha http://t.co/S0XWGfLJ9C&#8221; &#128588;
7105     @sweetakin Only rich white liberals know what's best for black people. If they don't see that, they're obviously Uncle Toms.                                    
8813     Don't mind black people.....hate niggers!                                                                                                                       
10198    I don't feel like all black people are niggers but if you're black and you're being a nigger, imma tell you, idc                                                
10499    I hate when black people try and be white so bad ... Your a nigger                                                                           

Before we go further, we'll build a quick function to remove the nonletters from the text.

In [225]:
def clean_tweet(tweet):
    # Remove nonletters
    text_only = re.sub("[^a-zA-Z]", " ", tweet)
    # Make it all lower case
    lower = text_only.lower()
    # Delete extra spaces
    single_spaces = ' '.join(lower.split())
    return single_spaces

In [226]:
all_tweets = df['tweet']
clean_df = all_tweets.apply(clean_tweet)

We don't want to switch all instances of "black" to "white" (and vice versa) because there are plenty of instances of those words that don't refer to people. To get around this I am making two lists - `black_phrases` and `white_phrases`. These are made by searching for all instances of "black" and "white" that actually refer to people, and putting that phrase in the list and the corresponding phrase (e.g. "black man" and "white man") in the opposite list. This will ensure that the nth entry in one list corresponds to the the nth entry in the other list.

In [160]:
# phrases = ['black people', 'white people', 'black girl', 'white girl', 'nigger', 'nigga', 'cracker', 'cracka', 'whiggers',
#          'white bitch', 'black bitch', 'white hoes', 'black hoes', 'white trash', 'black trash', 'white boy', 'black boy',
#          'black man', 'white man', 'black men', 'white men', 'white woman', 'black woman', 'white women', 'black women',
#         'whitey', 'blacky', 'black ass', 'white ass', 'black fag', 'white fag', 'black feminist', 'white feminist',
#         'black dude', 'white dude']

In [228]:
black_phrases = ['black people', 'black girl', 'nigger', 'nigga', 'black bitch',  'black hoes', 'black trash',
                 'black boy', 'black man',  'black men', 'black woman', 'black women', 'blacky',
                 'black ass', 'black fag','black feminist', 'black dude', 'black lady']
white_phrases = ['white people', 'white girl', 'cracker', 'cracka', 'white bitch',  'white hoes',  'white trash',
                 'white boy', 'white man',  'white men', 'white woman',  'white women', 'whitey',
                 'white ass', 'white fag', 'white feminist', 'white dude', 'white lady']

In [229]:
all_phrases = black_phrases + white_phrases

Now I need to find all the tweets that contain the words "black" or "white" but don't contain any of the words in `all_phrases`. I ran this search over and over again, each time adding another phrase to `black_phrases` and `white_phrases` until I felt like I got most of them.

In [230]:
clean_df[(clean_df.str.contains("black") | clean_df.str.contains("white")) & ~clean_df.str.contains('|'.join(words))]

19       black bottle amp a bad bitch                                                                                                                
83       blackchiquitita wow rt thatmanpalmer i m lost are those buttcheek piercings http t co yn guyouq yeah she s a hoe                            
84       blacknerdjade ok sis she d rather be a broke bitch shrugs she ll have to tell me how it works for her                                       
85       blackman tide whalelookyhere howdydowdy queer gaywad                                                                                        
89       cb baby white thunduh alsarabsss hes a beaner smh you can tell hes a mexican                                                                
97       celeynichole white thunduh how come you never bring me food i dont have a car retard                                                        
106      crhedrys pussy licking pussy meow meow stopwhitepeople https t co keegdcjs k               

In [231]:
#df[(df['tweet'].str.contains("black") | df['tweet'].str.contains("white")) & ~df['tweet'].str.contains('|'.join(all_phrases))]['tweet']

Now go through the data set and add new data

Now we'll go through the df and each time we see a black phrase we'll add a version with the white phrase. We'll make a new df to add the phrases to

In [237]:
# Create an empty dataframe with the same columns
white_phrase_df = df[0:0]
for x in range(len(df)):
    for y in range(len(black_phrases)):
        if black_phrases[y] in df.loc[x]['tweet']:
            #print(df.loc[x]['tweet'])
            white_phrase_draft = df.loc[x]
            # Switch out the black phrase for the equivalent white phrase
            temp_tweet = white_phrase_draft['tweet'].replace(black_phrases[y], white_phrases[y])
            # Insert the new tweet
            white_phrase_draft['tweet'] = temp_tweet
            # Add the phrase to the end of the white phrase df
            white_phrase_df.loc[(len(white_phrase_df))] = white_phrase_draft

C:\Users\HMISYS\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\HMISYS\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Now let's do the same thing except flip the race roles

In [238]:
# Create an empty dataframe with the same columns
black_phrase_df = df[0:0]
for x in range(len(df)):
    for y in range(len(white_phrases)):
        if white_phrases[y] in df.loc[x]['tweet']:
            #print(df.loc[x]['tweet'])
            black_phrase_draft = df.loc[x]
            # Switch out the black phrase for the equivalent white phrase
            temp_tweet = black_phrase_draft['tweet'].replace(white_phrases[y], black_phrases[y])
            # Insert the new tweet
            black_phrase_draft['tweet'] = temp_tweet
            # Add the phrase to the end of the white phrase df
            black_phrase_df.loc[(len(black_phrase_df))] = black_phrase_draft

C:\Users\HMISYS\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\HMISYS\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [239]:
white_phrase_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,15,3,0,3,0,1,""" bitch cracka miss me with it """
1,32,3,0,3,0,1,""" if you aint bout that Murder Game pussy cracka shut up """
2,49,3,1,2,0,1,""" these hoes like crackas that spend money not talk bout it """
3,50,3,1,2,0,1,""" we dont trust these crackas all these bitches """
4,51,3,0,3,0,1,""" yall crackas b cuffing hoes cause yall aint never have bitches """


Let's see how many instances we added

In [240]:
print(len(white_phrase_df))
print(len(black_phrase_df))

2318
365


For example, here's a newly created phrase:

In [214]:
white_phrase_df[white_phrase_df['Unnamed: 0'] == 15]

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,15,3,0,3,0,1,""" bitch cracka miss me with it """


And here's the original:

In [217]:
df[df['Unnamed: 0'] == 15]

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
15,15,3,0,3,0,1,""" bitch nigga miss me with it """


Now append them to the original data frame

In [245]:
df = df.append(white_phrase_df)
df = df.append(black_phrase_df)

In [246]:
len(df)

27466

We're started with 24783 samples and now have 27466.